In [22]:
import torch

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 9)

In [2]:
%%capture
# Colab에서 torch 2.2.1을 사용하고 있으므로, 패키지 충돌을 방지하기 위해 별도로 설치해야 합니다.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용하세요.
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass


## Unsloth


In [23]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024 # 최대 시퀀스 길이를 설정
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


이제 LoRA 어댑터를 추가하여 모든 파라미터 중 단 1% ~ 10%의 파라미터만 업데이트하면 됩니다!


In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### 데이터 준비


In [25]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


dataset = load_dataset("nooynoos/M.O.M_Dataset_GemmaSprint", split="train")

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

In [26]:
import os

# 환경 변수 설정
os.environ["TORCH_LOGS"] = "+dynamo"
os.environ["TORCHDYNAMO_VERBOSE"] = "1"

### 모델 훈련하기

이제 Huggingface TRL의 `SFTTrainer`를 사용해 봅시다!

- 참고 문서: [TRL SFT 문서](https://huggingface.co/docs/trl/sft_trainer)


In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [28]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA GeForce RTX 4070. Max memory = 11.994 GB.
7.086 GB of memory reserved.


In [29]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 600 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,2.717600
2,2.665700
3,2.606700
4,2.560600
5,2.270800
6,1.980600
7,1.807000
8,1.488000
9,1.485600
10,1.249300


PyTorch를 사용하여 모델 훈련 시 메모리 사용량과 훈련 시간을 계산하고 출력하는 코드입니다.

- `torch.cuda.max_memory_reserved()`를 사용하여 훈련 중에 예약된 최대 GPU 메모리를 계산합니다.
- 훈련 시작 시점의 GPU 메모리 사용량과 비교하여 LoRA(Low-Rank Adaptation)를 위해 사용된 추가 메모리 양을 계산합니다.
- 전체 GPU 메모리 대비 사용된 메모리의 비율을 계산합니다.
- 훈련에 소요된 총 시간을 초와 분 단위로 출력합니다.
- 예약된 최대 메모리, LoRA를 위해 사용된 메모리, 그리고 이들이 전체 GPU 메모리 대비 차지하는 비율을 출력합니다.


In [30]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

95.9918 seconds used for training.
1.6 minutes used for training.
Peak reserved memory = 9.377 GB.
Peak reserved memory for training = 2.291 GB.
Peak reserved memory % of max memory = 78.181 %.
Peak reserved memory for training % of max memory = 19.101 %.


### 추론

모델을 실행해 봅시다! 과연 나에게 잔소리를 해주는지!


In [31]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

(예시 1)

In [38]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "운동, 코딩, 과제",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
운동, 코딩, 과제

### Response:
운동은 언제 할 거니? 건강이 제일 중요하잖아! 그리고 코딩도 좀 해, 요즘 IT 시대에 코딩은 필수야. 과제도 중요하지만, 운동하고 코딩 조금씩 해두면 나중에 더 여유롭게 과제 풀 수 있지 않겠어? 하루에 조금씩이라도 해보자!<eos>


(예시2)

In [33]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "코세라 강의, 방 청소",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
코세라 강의, 방 청소

### Response:
코세라 강의 듣고 나서 방 청소도 해야지! 강의 듣는 동안에 방이 어질러져 있으면 집중이 안 될 거야. 청소하면서 깨끗한 방에서 코세라 강의 들으면 더 잘 들릴 거야. 엄마는 네가 깨끗한 방에서 좋은 강의 들을 수 있으면 좋겠어.<eos>


(예시 3)

In [40]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "일기 쓰기, 미리 계획 세우기",  # 지시사항
            "",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
일기 쓰기, 미리 계획 세우기

### Response:
일기 쓰는 건 언제 할 거야? 미리 계획 세우면 더 잘 할 수 있잖아! 일기 쓰면서 미리 계획 세우면 더 멋진 일이 될 거야. 엄마는 네가 계획적으로 행동하는 모습이 참 좋아해!<eos>


In [41]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "대학교 출석, 음식 줄이기",  # 지시사항
            "",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
대학교 출석, 음식 줄이기

### Response:
대학교 출석은 언제 할 거야? 돈도 벌고, 멋진 사람이 되고 싶잖아. 그리고 음식 줄이면서 체중도 늘리지 않아야지. 대학교에 가서 몸도 챙기고, 머리도 똑똑하게 해야지. 엄마는 네가 멋진 사람이 되길 바란단다!<eos>


In [42]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "취업 준비, 코세라 강의, 부트캠프",  # 지시사항
            "",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
취업 준비, 코세라 강의, 부트캠프

### Response:
취업 준비는 잘 되고 있니? 코세라 강의도 듣고, 부트캠프도 좀 들어야지. 취업 준비하면서 코세라 강의도 들고, 부트캠프도 틈틈이 들어봐. 그러면 네가 원하는 직장에 들어가는 기회도 늘고, 코세라 강의도 더 잘 들을 수 있을 거야. 엄마는 네가 행복하게 일할 수 있길 바란단다!<eos>


In [27]:
model.save_pretrained("gemma2-2b-M.O.M-gemma-sprint")  # 모델을 로컬에 저장합니다.
# model.push_to_hub("your_name/lora_model", token = "...") # 모델을 온라인 허브에 저장합니다.

만약 우리가 저장한 LoRA 어댑터를 추론을 위해 불러오고 싶다면, `False`를 `True`로 설정하세요.


### VLLM을 위한 float16 저장

우리는 `float16`으로 직접 저장하는 것을 지원합니다. `float16`을 위해서는 `merged_16bit`를 선택하거나, `int4`를 위해서는 `merged_4bit`를 선택하세요. 또한, 대체 방안으로 `lora` 어댑터를 사용할 수 있습니다.

모델을 저장하고 Hugging Face Hub에 푸시하는 다양한 방법을 보여주는 코드입니다.

- 16비트와 4비트 병합 방식으로 모델을 저장하고 푸시하는 조건문이 있으나, 이들은 실행되지 않도록 설정되어 있습니다.
- `model.save_pretrained_merged` 함수와 `model.push_to_hub_merged` 함수를 사용하여, "beomi/Llama-3-Open-Ko-8B" 모델을 "merged_4bit_forced" 방식으로 저장하고, "teddylee777/Llama-3-Open-Ko-8B-teddynote"로 Hugging Face Hub에 푸시합니다.
- LoRA 어댑터를 사용하여 모델을 저장하고 푸시하는 코드도 포함되어 있으나, 이 또한 실행되지 않도록 설정되어 있습니다.
- 모든 저장 및 푸시 작업에는 `tokenizer`와 특정 `save_method`가 필요하며, 푸시 작업에는 추가적으로 `token`이 필요합니다.


저장

In [28]:
base_model = "unsloth/gemma-2-2b"  # 병합을 수행할 베이스 모델
huggingface_token = ""  # HuggingFace 토큰
huggingface_repo = "gemma2-2b-M.O.M-gemma-sprint"  # 모델을 업로드할 repository
save_method = (
    "merged_16bit"  # "merged_4bit", "merged_4bit_forced", "merged_16bit", "lora"
)

### 옵션 1) 로컬에 저장

In [29]:
model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,  # 저장 방식을 16비트 병합으로 설정
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.49 out of 15.48 RAM for saving.


100%|██████████| 26/26 [00:00<00:00, 44.78it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


### 옵션 2) HuggingFace 에 업로드

In [30]:
# Hub 에 업로드
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token=huggingface_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.46 out of 15.48 RAM for saving.


100%|██████████| 26/26 [00:00<00:00, 55.59it/s]


Unsloth: Saving to organization with address nooynoos/gemma2-2b-M.O.M-gemma-sprint
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving to organization with address nooynoos/gemma2-2b-M.O.M-gemma-sprint
Unsloth: Uploading all files... Please wait...


model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/gemma2-2b-M.O.M-gemma-sprint
